In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
# explore dataset first
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))
# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)


comments
full
full_201510
stories


In [5]:
table_ref = dataset_ref.table("full")
table = client.get_table(table_ref)
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ()),
 SchemaField(

In [6]:
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,Wrecker – A HTTP Request Builder Written in Go,https://github.com/BrandonRomano/wrecker,None,None,BrandonRRomano,3.0,1434564076,2015-06-17 18:01:16+00:00,story,9733936,NaN,0.0,None,None
1,None,None,OTOH code completion is still only mediocre fo...,None,swah,NaN,1447097506,2015-11-09 19:31:46+00:00,comment,10535217,10532957.0,NaN,None,None
2,None,None,Are you saying the &quot;code generation layer...,None,micahbright,NaN,1462486065,2016-05-05 22:07:45+00:00,comment,11640016,11639332.0,NaN,None,None
3,None,None,The logic behind quoting FEMA is that the figu...,None,m52go,NaN,1507771783,2017-10-12 01:29:43+00:00,comment,15455008,15454472.0,NaN,None,None
4,None,None,I should clarify that I don't see technologica...,None,angusiguess,NaN,1329852926,2012-02-21 19:35:26+00:00,comment,3617742,3617076.0,NaN,None,None


In [7]:
query = """
        SELECT MAX(timestamp) as latestTimestamp
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type ='story'
        """

In [8]:
# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 374537619 bytes.


In [9]:
query_job = client.query(query)

In [10]:
query_job.to_dataframe()
# we see latest time is 2020-03-03

,latestTimestamp
0,2020-03-03 10:06:07+00:00


In [11]:
query = """
        SELECT title,text,timestamp, url, score, dead, deleted
        FROM `bigquery-public-data.hacker_news.full`
        WHERE timestamp BETWEEN '2018-06-01 00:00:00' AND '2019-06-01 00:00:00' AND type ='story'
        ORDER BY time
        """

In [12]:
# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 8345765911 bytes.


In [13]:
query = """
        SELECT tiele,text,timestamp, url, score, dead, deleted
        FROM `bigquery-public-data.hacker_news.full`
        WHERE timestamp BETWEEN '2013-06-01 00:00:00' AND '2019-12-30 00:00:00' AND type ='story'
        ORDER BY time
        """

In [14]:
query_job = client.query(query)
story6y = query_job.to_dataframe()

In [15]:
story6y.to_csv('6yn.csv')

In [16]:
story6y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344436 entries, 0 to 2344435
Data columns (total 7 columns):
 #   Column     Dtype              
---  ------     -----              
 0   title      object             
 1   text       object             
 2   timestamp  datetime64[ns, UTC]
 3   url        object             
 4   score      float64            
 5   dead       object             
 6   deleted    object             
dtypes: datetime64[ns, UTC](1), float64(1), object(5)
memory usage: 125.2+ MB


In [17]:
story6y.head(10)

,title,text,timestamp,url,score,dead,deleted
0,How to Not Mess Up Your Tech Startup,None,2013-06-01 00:01:56+00:00,http://kristinabjoran.com/how-to-not-mess-up-y...,2.0,None,None
1,None,None,2013-06-01 00:03:32+00:00,None,NaN,True,True
2,Someday you may ditch your two-factor authenti...,None,2013-06-01 00:04:34+00:00,http://arstechnica.com/security/2013/05/someda...,1.0,None,None
3,None,None,2013-06-01 00:05:35+00:00,None,NaN,True,True
4,Will State.com become the world's opinion forum?,None,2013-06-01 00:05:54+00:00,http://pandodaily.com/2013/05/31/making-sense-...,2.0,None,None
5,LightUp Teaches Kids Electronics With Augmente...,None,2013-06-01 00:06:49+00:00,http://techcrunch.com/2013/05/31/lightup-helps...,16.0,None,None
6,Getting Things Done: Why GTD for Sales is the ...,None,2013-06-01 00:07:12+00:00,http://blog.voltagecrm.com/getting-things-done...,1.0,None,None
7,Never let that fire go out,None,2013-06-01 00:12:28+00:00,https://medium.com/advice-to-graduates/8c7bcb3...,1.0,None,None
8,Ask HN: Where/how to get the 'typical startup...,Are all startup websites based on a certain th...,2013-06-01 00:15:44+00:00,None,2.0,None,None
9,Rest.li a framework from LinkedIn for building...,None,2013-06-01 00:18:40+00:00,http://rest.li/,3.0,None,None


In [18]:
# replace non values with nan
story6y.fillna(value=np.nan, inplace=True)
# filter out delted stories
story6y_f = story6y[story6y.deleted.isnull()]

In [19]:
# build the series of content, ignoring nan values
stories = story6y_f.title.fillna(' ').astype(str) + ' ' + story6y_f.text.fillna(' ').astype(str)

In [135]:
stories

0                     How to Not Mess Up Your Tech Startup  
2          Someday you may ditch your two-factor authenti...
4           Will State.com become the world's opinion for...
5          LightUp Teaches Kids Electronics With Augmente...
6          Getting Things Done: Why GTD for Sales is the ...
                                 ...                        
2128596    Credit Score The best credit repair is from es...
2128598    Solving Tech's Ethics Problem Could Start in t...
2128599        “Why” and “How” TrueCar Moved to True CI/CD  
2128600    Undesign: Collection of free design tools and ...
2128601    Bismarck vs. Beveridge: is there increasing co...
Length: 2036083, dtype: object

In [156]:
# extract domain as an additional feature
import tldextract
domains = story6y_f.url.fillna(' ')

In [158]:
maindomain = domains.apply(lambda x: tldextract.extract(x).domain)

In [221]:
top15sites = maindomain.value_counts()[1:16]

In [222]:
top15sites

github         94921
medium         83323
youtube        50240
nytimes        35567
techcrunch     25234
blogspot       22677
bbc            20582
theguardian    17610
arstechnica    17524
google         16807
bloomberg      15419
wikipedia      14081
theverge       13611
wired          13046
wsj            12556
Name: url, dtype: int64

In [160]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [61]:
# text prepprocessing
lemmatizer=WordNetLemmatizer() #For words Lemmatization
stop_words=set(stopwords.words('english'))

In [171]:
# text processing
# 1. convert to lower case, clean text of all punctuations
# 2. remove stop words
# 3. lemmatize

def process_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = ' '.join([lemmatizer.lemmatize(t) for t in tokenized_text if t not in stop_words and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)])
    return cleaned_text


In [182]:
stories_clean = stories.apply(lambda x: process_text(x))

In [ ]:
top15words = stories_clean.apply(lambda x: x.split()).value_counts()[1:16]

In [1]:
stories_clean

NameError: name 'stories_clean' is not defined

In [186]:
# add main domain name back to stories
stories_aug = stories_clean + ' ' + maindomain

In [191]:
stories_aug

0                           mess tech startup kristinabjoran
2          someday may ditch two-factor authenticator ele...
4            state.com become world opinion forum pandodaily
5          lightup teach kid electronics augmented realit...
6              getting thing done gtd sale answer voltagecrm
                                 ...                        
2128596    credit score best credit repair esaycredit pay...
2128598    solving tech ethic problem could start classro...
2128599                            truecar moved true medium
2128600    undesign collection free design tool resource ...
2128601    bismarck beveridge increasing convergence heal...
Length: 2036083, dtype: object

In [197]:
# if using genism, need the tokenzied form for input
stories_token = stories_aug.apply(lambda x: x.split())

In [199]:
from gensim import corpora
dictionary = corpora.Dictionary(stories_token)
corpus = [dictionary.doc2bow(text) for text in stories_token]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')


In [202]:
import gensim
# assume we have 15 topics
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 15, id2word=dictionary, passes=5)
ldamodel.save('hn15.gensim')


In [204]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.078*"youtube" + 0.051*"video" + 0.040*"game" + 0.024*"model" + 0.023*"news" + 0.019*"car" + 0.019*"youtu" + 0.017*"hacker" + 0.017*"wired" + 0.017*"tesla"')
(1, '0.033*"china" + 0.023*"launch" + 0.022*"phone" + 0.020*"buy" + 0.020*"market" + 0.019*"research" + 0.017*"remote" + 0.017*"tip" + 0.015*"find" + 0.015*"call"')
(2, '0.040*"build" + 0.039*"design" + 0.023*"part" + 0.022*"building" + 0.021*"api" + 0.021*"image" + 0.021*"javascript" + 0.020*"interview" + 0.020*"product" + 0.019*"process"')
(3, '0.108*"google" + 0.041*"service" + 0.035*"network" + 0.032*"user" + 0.025*"guide" + 0.024*"search" + 0.021*"reddit" + 0.018*"key" + 0.018*"city" + 0.013*"black"')
(4, '0.162*"medium" + 0.076*"data" + 0.050*"nytimes" + 0.034*"pdf" + 0.025*"theverge" + 0.022*"cloud" + 0.021*"social" + 0.016*"uber" + 0.016*"big" + 0.016*"privacy"')
(5, '0.029*"ask" + 0.020*"like" + 0.020*"quot" + 0.016*"work" + 0.015*"would" + 0.014*"people" + 0.013*"get" + 0.012*"one" + 0.012*"know" + 0.011*"want"')
(

In [205]:
# visualize

import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

D:\python3\envs\tdi\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [206]:
pyLDAvis.save_html(lda_display, 'lda.html')